In [76]:
#import necessary libraries
from flask import Flask
from flask_restful import Resource, Api, reqparse
from google.cloud import bigquery
import pandas_gbq
import pandas
import json
import os

In [75]:
#google credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Hurricane -32cb24436d46.json"

# #create Flass app instance
# app = Flask(__name__)
# api = Api(app)


#setup connection to BigAssQuery
client = bigquery.Client()


# @app.route("/")
# def index():

query = """
    SELECT year, latitude, longitude, sea_surface_temp
    FROM `bigquery-public-data.noaa_icoads.icoads_core_*` 
    WHERE year >=1851 
    AND latitude <=50
    AND latitude >=0
    AND longitude <=-50
    AND longitude >=-100
    ORDER BY year
"""

# query_res = client.query(query) 

In [79]:
# query_res

In [80]:
import pandas as pd
import pandas_gbq
df = pandas_gbq.read_gbq(query)


Downloading:  10%|████▉                                            | 16978374/169220630 [10:21<1:31:43, 27661.80rows/s]


Downloading:  21%|██████████▎                                      | 35765473/169220630 [21:29<1:19:46, 27880.91rows/s]


Downloading:  32%|███████████████▉                                 | 54860547/169220630 [32:30<1:06:40, 28585.59rows/s]


Downloading:  43%|██████████████████████▏                            | 73605082/169220630 [43:32<50:49, 31350.37rows/s]


Downloading:  54%|███████████████████████████▌                       | 91591833/169220630 [54:04<49:05, 26359.44rows/s]


Downloading:  64%|██████████████████████████████▉                 | 109018134/169220630 [1:04:41<36:40, 27353.66rows/s]


Downloading:  76%|████████████████████████████████████▍           | 128324378/169220630 [1:15:54<23:16, 29276.70rows/s]


Downloading:  87%|█████████████████████████████████████████▋      | 146991330/169220630 [1:26:56<13:31, 27377.45rows/s]


Downloading:  98%|██████████████████████████████████████████████▉ | 165367538/169220630 [1:37:39<02:26, 26343.02rows/s]


Downloading: 100%|████████████████████████████████████████████████| 169220630/169220630 [1:39:55<00:00, 28223.64rows/s]


In [81]:
lat_bins=range(0, 51, 10)
df['latitudeSquare'] = pd.cut(df['latitude'], bins=lat_bins)

In [82]:
long_bins=range(-100, -49, 10)
df['longitudeSquare']=pd.cut(df['longitude'], bins=long_bins)

In [83]:
dfSquare=df.groupby(['year','latitudeSquare','longitudeSquare']).agg({'sea_surface_temp': ['min','max','mean','median']})
dfSquare.head()

sea_surface_temp                        
                                                 min   max       mean median
year latitudeSquare longitudeSquare                                         
1851 (0, 10]        (-100, -90]                 19.4  26.7  23.557895   23.3
                    (-90, -80]                  21.7  27.8  25.121739   24.4
                    (-80, -70]                  25.0  28.9  26.983333   26.4
                    (-70, -60]                   NaN   NaN        NaN    NaN
                    (-60, -50]                  26.1  28.3  26.800000   26.4

In [84]:
dfSquare.to_json(r'yearData.json')

In [85]:
dfProcessed = dfSquare.reset_index()
dfProcessed

year latitudeSquare longitudeSquare sea_surface_temp                   \
                                                       min   max       mean   
0     1851        (0, 10]     (-100, -90]             19.4  26.7  23.557895   
1     1851        (0, 10]      (-90, -80]             21.7  27.8  25.121739   
2     1851        (0, 10]      (-80, -70]             25.0  28.9  26.983333   
3     1851        (0, 10]      (-70, -60]              NaN   NaN        NaN   
4     1851        (0, 10]      (-60, -50]             26.1  28.3  26.800000   
...    ...            ...             ...              ...   ...        ...   
4170  2017       (40, 50]     (-100, -90]             -0.5  24.6  12.544084   
4171  2017       (40, 50]      (-90, -80]             -2.5  29.0  15.355935   
4172  2017       (40, 50]      (-80, -70]             -1.5  29.4  11.725217   
4173  2017       (40, 50]      (-70, -60]             -1.8  28.7   7.306804   
4174  2017       (40, 50]      (-60, -50]             -2.0  30.0   5.892106   

             
     median  
0      23.3  
1      24.4  
2      26.4  
3       NaN  
4      26.4  
...     ...  
4170   13.2  
4171   17.0  
4172   10.4  
4173    6.2  
4174    4.2  

[4175 rows x 7 columns]

In [92]:
dfProcessed.to_json(path_or_buf="sstColumns.json",orient="columns", index="false")
